<a href="https://colab.research.google.com/github/rafiulislamri/Deep-CNN-weather_forecast_Bangladesh_v2.0-Advanced-Hybrid-DLM-------Training-08/blob/main/weather_forecast_Bangladesh_v2_0(Advanced_Hybrid_DLM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Architecture: Hybrid Conv1D + LSTM + Attention Mechanism

In [ ]:
!pip install plotly seaborn scikit-plot -q

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import os
import warnings
warnings.filterwarnings('ignore')

# Deep Learning imports
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (LSTM, Dense, Dropout, BatchNormalization,
                                   Conv1D, MaxPooling1D, GlobalAveragePooling1D,
                                   Input, MultiHeadAttention, LayerNormalization,
                                   Flatten, Reshape, concatenate)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.utils import plot_model

# Sklearn imports
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.19.0


In [ ]:
from google.colab import files

uploaded = files.upload()

for filename in uploaded.keys():
    print(f'User uploaded file "{filename}"')
    # Assuming the uploaded file is the dataset you want to use
    df = pd.read_csv(filename)

# Display the first few rows of the uploaded dataset
print("\nFirst 5 rows of the uploaded dataset:")
display(df.head())

print(f"\nDataset shape: {df.shape}")
print(f"Dataset columns: {df.columns.tolist()}")

Saving BD_weather.csv to BD_weather.csv
User uploaded file "BD_weather.csv"

First 5 rows of the uploaded dataset:


,Station,Year,Month,Day,Rainfall,Sunshine,Humidity,Temperature
0,Ambaganctg,2008,1,1,0.0,9.1,78.0,20.1
1,Ambaganctg,2008,1,2,0.0,5.6,79.0,21.0
2,Ambaganctg,2008,1,3,0.0,8.6,82.0,20.1
3,Ambaganctg,2008,1,4,0.0,9.1,73.0,21.0
4,Ambaganctg,2008,1,5,0.0,9.4,67.0,20.3



Dataset shape: (543839, 8)
Dataset columns: ['Station', 'Year', 'Month', 'Day', 'Rainfall', 'Sunshine', 'Humidity', 'Temperature']


In [ ]:
# Data Exploration and Cleaning
print("\n" + "="*50)
print("DATA EXPLORATION & CLEANING")
print("="*50)

# Display basic information
print("\nDataset Info:")
print(df.info())
print(f"\nMissing values:\n{df.isnull().sum()}")

# Display first few rows
print("\nFirst 5 rows:")
print(df.head())

# Check for date-related columns and standardize
# Combine Year, Month, and Day columns to create a datetime column
try:
    df['date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
    print("\n📅 Created 'date' column from Year, Month, Day")
except Exception as e:
    print(f"\n❌ Could not create 'date' column from Year, Month, Day: {e}")
    # Fallback if combining fails (though it should work with these columns)
    print("\n❌ Falling back to creating date index...")
    try:
        df['date'] = pd.date_range(start='2008-01-01', periods=len(df), freq='D')
        print("\n📅 Created date index starting from 2008-01-01")
    except Exception as date_range_error:
         print(f"\n❌ Failed to create date index: {date_range_error}")


# Sort by date
df = df.sort_values('date').reset_index(drop=True)

# Handle missing values
print(f"\n🔄 Handling missing values...")
initial_shape = df.shape
df = df.dropna(subset=['date'])  # Remove rows with invalid dates

# Fill numeric columns with forward fill
numeric_columns = df.select_dtypes(include=[np.number]).columns
df[numeric_columns] = df[numeric_columns].fillna(method='ffill')

print(f"Removed {initial_shape[0] - df.shape[0]} rows with invalid dates")
print(f"Final dataset shape after cleaning: {df.shape}")

# Create time-based features
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day_of_year'] = df['date'].dt.dayofyear
df['day_of_week'] = df['date'].dt.dayofweek

print(f"\n✅ Data cleaning completed!")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")


DATA EXPLORATION & CLEANING

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 543839 entries, 0 to 543838
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Station      543839 non-null  object 
 1   Year         543839 non-null  int64  
 2   Month        543839 non-null  int64  
 3   Day          543839 non-null  int64  
 4   Rainfall     543839 non-null  float64
 5   Sunshine     543839 non-null  float64
 6   Humidity     543839 non-null  float64
 7   Temperature  543839 non-null  float64
dtypes: float64(4), int64(3), object(1)
memory usage: 33.2+ MB
None

Missing values:
Station        0
Year           0
Month          0
Day            0
Rainfall       0
Sunshine       0
Humidity       0
Temperature    0
dtype: int64

First 5 rows:
      Station  Year  Month  Day  Rainfall  Sunshine  Humidity  Temperature
0  Ambaganctg  2008      1    1       0.0       9.1      78.0         20.1
1  Ambaganctg  20

In [ ]:
# Comprehensive Data Visualization
print("\n" + "="*50)
print("DATA VISUALIZATION")
print("="*50)

# Identify numeric columns for visualization
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(f"Available numeric columns: {numeric_cols}")

# Remove date-related numeric columns from visualization
exclude_cols = ['year', 'month', 'day_of_year', 'day_of_week']
plot_cols = [col for col in numeric_cols if col not in exclude_cols]

print(f"Columns for visualization: {plot_cols}")

# Create comprehensive visualizations
if plot_cols:
    plt.figure(figsize=(20, 15))

    # Plot distributions
    for i, col in enumerate(plot_cols[:6]):  # Plot first 6 numeric columns
        plt.subplot(3, 3, i+1)
        if df[col].nunique() > 1:  # Only plot if there's variation
            sns.histplot(df[col], kde=True, bins=30)
            plt.title(f'Distribution of {col}')
            plt.xlabel(col)
        else:
            plt.text(0.5, 0.5, f'No variation in {col}',
                    horizontalalignment='center', verticalalignment='center',
                    transform=plt.gca().transAxes)
            plt.axis('off')

    # Time series plot for first numeric column
    if len(plot_cols) > 0:
        plt.subplot(3, 3, 7)
        sample_df = df.iloc[::max(1, len(df)//1000)]  # Sample for clarity
        plt.plot(sample_df['date'], sample_df[plot_cols[0]], alpha=0.7)
        plt.title(f'{plot_cols[0]} Over Time')
        plt.xlabel('Date')
        plt.ylabel(plot_cols[0])
        plt.xticks(rotation=45)

    # Correlation heatmap
    if len(plot_cols) > 1:
        plt.subplot(3, 3, 8)
        correlation_matrix = df[plot_cols].corr()
        sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
        plt.title('Feature Correlations')

    # Monthly patterns for first numeric column
    if len(plot_cols) > 0:
        plt.subplot(3, 3, 9)
        monthly_avg = df.groupby('month')[plot_cols[0]].mean()
        plt.plot(monthly_avg.index, monthly_avg.values, marker='o')
        plt.title(f'Monthly {plot_cols[0]} Pattern')
        plt.xlabel('Month')
        plt.ylabel(plot_cols[0])

    plt.tight_layout()
    plt.show()

# Interactive time series plot
if len(plot_cols) > 0:
    fig = go.Figure()

    for col in plot_cols[:3]:  # Plot first 3 columns
        fig.add_trace(go.Scatter(
            x=df['date'],
            y=df[col],
            name=col,
            opacity=0.7
        ))

    fig.update_layout(
        title='Weather Parameters Over Time',
        xaxis_title='Date',
        yaxis_title='Values',
        height=500
    )
    fig.show()

In [ ]:
# Advanced Feature Engineering
print("\n" + "="*50)
print("FEATURE ENGINEERING")
print("="*50)

# Select target variable (choose the first suitable numeric column)
potential_targets = [col for col in plot_cols if df[col].nunique() > 10]  # Columns with sufficient variation
if potential_targets:
    target_column = potential_targets[0]
    print(f"Selected target variable: {target_column}")
else:
    target_column = numeric_cols[0] if numeric_cols else 'temperature'
    print(f"Using default target: {target_column}")

# Create cyclical features for temporal patterns
df['day_sin'] = np.sin(2 * np.pi * df['day_of_year']/365)
df['day_cos'] = np.cos(2 * np.pi * df['day_of_year']/365)
df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
df['month_cos'] = np.cos(2 * np.pi * df['month']/12)

# Create lag features
if target_column in df.columns:
    for lag in [1, 2, 3, 7]:  # 1 day, 2 days, 3 days, 1 week
        df[f'{target_column}_lag_{lag}'] = df[target_column].shift(lag)

# Rolling statistics
if target_column in df.columns:
    for window in [3, 7, 14]:
        df[f'{target_column}_rolling_mean_{window}'] = df[target_column].rolling(window=window).mean()
        df[f'{target_column}_rolling_std_{window}'] = df[target_column].rolling(window=window).std()

# Handle categorical variables (stations)
if 'station' in df.columns:
    print(f"Stations found: {df['station'].unique()}")
    station_dummies = pd.get_dummies(df['station'], prefix='station')
    df = pd.concat([df, station_dummies], axis=1)

# Drop rows with NaN values created by lag features
initial_size = len(df)
df = df.dropna()
print(f"Removed {initial_size - len(df)} rows with NaN values after feature engineering")
print(f"Final dataset shape: {df.shape}")


FEATURE ENGINEERING
Selected target variable: Year
Removed 13 rows with NaN values after feature engineering
Final dataset shape: (543826, 27)


In [ ]:
# Data Preparation for Deep Learning
print("\n" + "="*50)
print("DATA PREPARATION FOR DEEP LEARNING")
print("="*50)

# Select features for modeling (exclude date and target-related columns)
exclude_features = ['date', 'year', 'month', 'day_of_year', 'day_of_week', 'data_type', 'source_file', target_column]
feature_columns = [col for col in df.columns if col not in exclude_features and col != target_column]

# Remove lag features of the target to avoid data leakage
feature_columns = [col for col in feature_columns if f'{target_column}_lag' not in col]

print(f"Selected {len(feature_columns)} features: {feature_columns}")

X = df[feature_columns].select_dtypes(include=[np.number])
y = df[target_column]

print(f"Feature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")

# Scale the features
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1)).flatten()

# Create sequences for LSTM
def create_sequences(X, y, time_steps=30):
    X_seq, y_seq = [], []
    for i in range(time_steps, len(X)):
        X_seq.append(X[i-time_steps:i])
        y_seq.append(y[i])
    return np.array(X_seq), np.array(y_seq)

TIME_STEPS = 30
X_sequences, y_sequences = create_sequences(X_scaled, y_scaled, TIME_STEPS)

print(f"Sequences shape: {X_sequences.shape}")
print(f"Target sequences shape: {y_sequences.shape}")

# Split into train/validation/test (maintaining temporal order)
split_idx1 = int(0.7 * len(X_sequences))
split_idx2 = int(0.85 * len(X_sequences))

X_train, y_train = X_sequences[:split_idx1], y_sequences[:split_idx1]
X_val, y_val = X_sequences[split_idx1:split_idx2], y_sequences[split_idx1:split_idx2]
X_test, y_test = X_sequences[split_idx2:], y_sequences[split_idx2:]

print(f"Training set: {X_train.shape}, {y_train.shape}")
print(f"Validation set: {X_val.shape}, {y_val.shape}")
print(f"Test set: {X_test.shape}, {y_test.shape}")


DATA PREPARATION FOR DEEP LEARNING
Selected 17 features: ['Station', 'Month', 'Day', 'Rainfall', 'Sunshine', 'Humidity', 'Temperature', 'day_sin', 'day_cos', 'month_sin', 'month_cos', 'Year_rolling_mean_3', 'Year_rolling_std_3', 'Year_rolling_mean_7', 'Year_rolling_std_7', 'Year_rolling_mean_14', 'Year_rolling_std_14']
Feature matrix shape: (543826, 16)
Target vector shape: (543826,)
Sequences shape: (543796, 30, 16)
Target sequences shape: (543796,)
Training set: (380657, 30, 16), (380657,)
Validation set: (81569, 30, 16), (81569,)
Test set: (81570, 30, 16), (81570,)


In [ ]:
# Build Advanced Hybrid Model (same as before)
print("\n" + "="*50)
print("BUILDING ADVANCED HYBRID MODEL")
print("="*50)

def create_hybrid_attention_model(input_shape):
    # Input layer
    inputs = Input(shape=input_shape)

    # 1D Convolutional layers for local pattern detection
    conv1 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(inputs)
    batch_norm1 = BatchNormalization()(conv1)
    pool1 = MaxPooling1D(pool_size=2)(batch_norm1)
    dropout1 = Dropout(0.2)(pool1)

    conv2 = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(dropout1)
    batch_norm2 = BatchNormalization()(conv2)
    pool2 = MaxPooling1D(pool_size=2)(batch_norm2)
    dropout2 = Dropout(0.2)(pool2)

    # LSTM layers for temporal dependencies
    lstm1 = LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(dropout2)
    batch_norm3 = BatchNormalization()(lstm1)

    lstm2 = LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(batch_norm3)
    batch_norm4 = BatchNormalization()(lstm2)

    # Multi-Head Attention mechanism
    attention_output = MultiHeadAttention(
        num_heads=8,
        key_dim=64,
        dropout=0.1
    )(batch_norm4, batch_norm4)

    # Residual connection with layer normalization
    residual = LayerNormalization()(attention_output + batch_norm4)

    # Global pooling and dense layers
    global_pool = GlobalAveragePooling1D()(residual)

    dense1 = Dense(128, activation='relu')(global_pool)
    dropout3 = Dropout(0.3)(dense1)
    batch_norm5 = BatchNormalization()(dropout3)

    dense2 = Dense(64, activation='relu')(batch_norm5)
    dropout4 = Dropout(0.3)(dense2)

    # Output layer
    outputs = Dense(1, activation='linear')(dropout4)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Create model
input_shape = (X_train.shape[1], X_train.shape[2])
model = create_hybrid_attention_model(input_shape)

# Compile model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

# Display model architecture
model.summary()


BUILDING ADVANCED HYBRID MODEL


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 30, 16)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 30, 64)    │      3,136 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 30, 64)    │        256 │ conv1d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d       │ (None, 15, 64)    │          0 │ batch_normalizat… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 15, 64)    │          0 │ max_pooling1d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 15, 128)   │     24,704 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 15, 128)   │        512 │ conv1d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_1     │ (None, 7, 128)    │          0 │ batch_normalizat… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 7, 128)    │          0 │ max_pooling1d_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 7, 128)    │    131,584 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 7, 128)    │        512 │ lstm[0][0]        │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 7, 64)     │     49,408 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 7, 64)     │        256 │ lstm_1[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 7, 64)     │    132,672 │ batch_normalizat… │
│ (MultiHeadAttentio… │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 7, 64)     │          0 │ multi_head_atten… │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 7, 64)     │        128 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 64)        │          0 │ layer_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │      8,320 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 128)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 360,321 (1.37 MB)

 Trainable params: 359,297 (1.37 MB)

 Non-trainable params: 1,024 (4.00 KB)

In [ ]:
# Train the Model (same as before)
print("\n" + "="*50)
print("TRAINING THE MODEL")
print("="*50)

# Callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=10,
    min_lr=0.0001,
    verbose=1
)

model_checkpoint = ModelCheckpoint(
    'best_weather_model.h5',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

# Train the model
history = model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=20,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, reduce_lr, model_checkpoint],
    verbose=1
)


TRAINING THE MODEL
Epoch 1/20
11895/11896 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.1379 - mae: 0.2373
Epoch 1: val_loss improved from inf to 0.11529, saving model to best_weather_model.h5


11896/11896 ━━━━━━━━━━━━━━━━━━━━ 732s 60ms/step - loss: 0.1379 - mae: 0.2373 - val_loss: 0.1153 - val_mae: 0.3198 - learning_rate: 0.0010
Epoch 2/20
11895/11896 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0207 - mae: 0.1051
Epoch 2: val_loss improved from 0.11529 to 0.05396, saving model to best_weather_model.h5


11896/11896 ━━━━━━━━━━━━━━━━━━━━ 725s 61ms/step - loss: 0.0207 - mae: 0.1051 - val_loss: 0.0540 - val_mae: 0.1982 - learning_rate: 0.0010
Epoch 3/20
11895/11896 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0172 - mae: 0.0948
Epoch 3: val_loss did not improve from 0.05396
11896/11896 ━━━━━━━━━━━━━━━━━━━━ 730s 61ms/step - loss: 0.0172 - mae: 0.0948 - val_loss: 0.1130 - val_mae: 0.3115 - learning_rate: 0.0010
Epoch 4/20
11895/11896 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.0158 - mae: 0.0906
Epoch 4: val_loss did not improve from 0.05396
11896/11896 ━━━━━━━━━━━━━━━━━━━━ 728s 61ms/step - loss: 0.0158 - mae: 0.0906 - val_loss: 0.0593 - val_mae: 0.2123 - learning_rate: 0.0010
Epoch 5/20
11895/11896 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.0149 - mae: 0.0876
Epoch 5: val_loss did not improve from 0.05396
11896/11896 ━━━━━━━━━━━━━━━━━━━━ 735s 62ms/step - loss: 0.0149 - mae: 0.0876 - val_loss: 0.1314 - val_mae: 0.3401 - learning_rate: 0.0010
Epoch 6/20
11895/11896 ━━━━━━━━━━━━━━━━━━━━ 0s 59

11896/11896 ━━━━━━━━━━━━━━━━━━━━ 742s 62ms/step - loss: 0.0137 - mae: 0.0834 - val_loss: 0.0516 - val_mae: 0.1927 - learning_rate: 0.0010
Epoch 11/20
11895/11896 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0135 - mae: 0.0828
Epoch 11: val_loss did not improve from 0.05159
11896/11896 ━━━━━━━━━━━━━━━━━━━━ 737s 61ms/step - loss: 0.0135 - mae: 0.0828 - val_loss: 0.0700 - val_mae: 0.2360 - learning_rate: 0.0010
Epoch 12/20
11895/11896 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0131 - mae: 0.0815
Epoch 12: val_loss did not improve from 0.05159
11896/11896 ━━━━━━━━━━━━━━━━━━━━ 730s 60ms/step - loss: 0.0131 - mae: 0.0815 - val_loss: 0.0581 - val_mae: 0.2110 - learning_rate: 0.0010
Epoch 13/20
11895/11896 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0134 - mae: 0.0820
Epoch 13: val_loss did not improve from 0.05159
11896/11896 ━━━━━━━━━━━━━━━━━━━━ 715s 60ms/step - loss: 0.0134 - mae: 0.0820 - val_loss: 0.0678 - val_mae: 0.2362 - learning_rate: 0.0010
Epoch 14/20
11896/11896 ━━━━━━━━━━━━━━━━━━━

In [ ]:
# Get user input for tomorrow's weather in Dhaka
print("\n" + "="*50)
print("TOMORROW'S WEATHER PREDICTION FOR DHAKA")
print("="*50)

print("Please enter tomorrow's weather details for Dhaka:")

# Define a dictionary to store user input for features used in training
tomorrow_data = {}

# Get input for each feature used in the model, excluding the target and engineered features
# that depend on future values (lags and rolling stats of the target)
input_features = [col for col in feature_columns if 'lag' not in col and 'rolling' not in col]

# Manually handle 'Station' as it was one-hot encoded
# We need to get the station index for Dhaka
try:
    dhaka_station_col = [col for col in df.columns if 'station_Dhaka' in col]
    if dhaka_station_col:
        dhaka_station_index = df.columns.get_loc(dhaka_station_col[0])
        # Create a row with zeros for all station dummies, then set Dhaka to 1
        station_input = np.zeros(len([col for col in df.columns if 'station_' in col]))
        # Find the index of 'station_Dhaka' in the dummy columns
        station_dummy_cols = [col for col in df.columns if 'station_' in col]
        try:
            dhaka_dummy_index = station_dummy_cols.index('station_Dhaka')
            station_input[dhaka_dummy_index] = 1
            print("\nAssuming 'Dhaka' station for prediction.")
        except ValueError:
            print("Warning: 'station_Dhaka' not found in dummy columns. Station information will not be included.")
            station_input = [] # No station info if not found
    else:
         print("Warning: Station information not found in features. Station information will not be included.")
         station_input = [] # No station info if not found

except Exception as e:
    print(f"Error handling station input: {e}")
    station_input = []

# Get input for numeric features
for col in input_features:
    if col != 'Station' and col in X.columns: # Exclude 'Station' here, handle separately
        while True:
            try:
                value = float(input(f"Enter value for {col}: "))
                tomorrow_data[col] = value
                break
            except ValueError:
                print("Invalid input. Please enter a numeric value.")

# Create a DataFrame for tomorrow's data
tomorrow_df = pd.DataFrame([tomorrow_data])

# Add station dummy columns
if len(station_input) > 0:
    station_dummy_df = pd.DataFrame([station_input], columns=station_dummy_cols)
    tomorrow_df = pd.concat([tomorrow_df, station_dummy_df], axis=1)

# Ensure all features used during training are present, fill missing with 0 (especially for stations)
for col in X.columns:
    if col not in tomorrow_df.columns:
        tomorrow_df[col] = 0

# Ensure the order of columns matches the training data
tomorrow_df = tomorrow_df[X.columns]

print("\nPrepared input data for prediction:")
display(tomorrow_df)


TOMORROW'S WEATHER PREDICTION FOR DHAKA
Please enter tomorrow's weather details for Dhaka:
Enter value for Month: November
Invalid input. Please enter a numeric value.
Enter value for Month: 11
Enter value for Day: 2
Enter value for Rainfall: 1
Enter value for Sunshine: 2
Enter value for Humidity: 3
Enter value for Temperature: 20
Enter value for day_sin: 1
Enter value for day_cos: 2
Enter value for month_sin: 1
Enter value for month_cos: 2

Prepared input data for prediction:


,Month,Day,Rainfall,Sunshine,Humidity,Temperature,day_sin,day_cos,month_sin,month_cos,Year_rolling_mean_3,Year_rolling_std_3,Year_rolling_mean_7,Year_rolling_std_7,Year_rolling_mean_14,Year_rolling_std_14
0,11.0,2.0,1.0,2.0,3.0,20.0,1.0,2.0,1.0,2.0,0,0,0,0,0,0


In [ ]:
# Save the model in .h5 format
try:
    model.save('weather_prediction_model.h5')
    print("\nModel saved successfully in weather_prediction_model.h5")
except Exception as e:
    print(f"\nError saving the model: {e}")


Model saved successfully in weather_prediction_model.h5


In [ ]:
# Scale the input data using the same scaler used for training
try:
    tomorrow_scaled = scaler_X.transform(tomorrow_df)
    print("\nScaled input data.")
except Exception as e:
    print(f"\nError scaling input data: {e}")
    tomorrow_scaled = None # Set to None to indicate failure

if tomorrow_scaled is not None:
    # Reshape for LSTM input (samples, time_steps, features)
    # Since we are predicting for one day, time_steps will be 1 for the single prediction
    # However, the model expects TIME_STEPS (30) as input sequence length.
    # To use the model, we need a sequence of the past TIME_STEPS days leading up to tomorrow.
    # This requires having access to the last TIME_STEPS days of data, which we don't
    # directly take as user input here.

    # A common approach for single-step prediction using sequence models is to provide
    # a sequence ending with the features of the day for which we want to predict.
    # Since we only have tomorrow's features from user input, we'll create a sequence
    # where all TIME_STEPS are the same as tomorrow's features. This is a simplification
    # and might not yield the best results compared to using actual past data.

    # Reshape the single sample to match the TIME_STEPS sequence length required by the model
    tomorrow_sequence = np.repeat(tomorrow_scaled[np.newaxis, :, :], TIME_STEPS, axis=1)

    print(f"Reshaped sequence for model input: {tomorrow_sequence.shape}")

    # Make the prediction
    try:
        predicted_scaled = model.predict(tomorrow_sequence)
        print("\nPrediction made.")

        # Inverse transform the prediction to the original scale
        predicted_original = scaler_y.inverse_transform(predicted_scaled)

        print("\n" + "="*50)
        print("PREDICTION RESULT")
        print("="*50)
        print(f"Predicted {target_column} for tomorrow in Dhaka: {predicted_original[0][0]:.2f}")

    except Exception as e:
        print(f"\nError during prediction or inverse transformation: {e}")
else:
    print("\nSkipping prediction due to scaling error.")


Scaled input data.
Reshaped sequence for model input: (1, 30, 16)
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step

Prediction made.

PREDICTION RESULT
Predicted Year for tomorrow in Dhaka: 1961.58
